## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [8]:
data.drop("date", axis=1, inplace=True)
data.drop("symbol", axis=1, inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
df = data.head(1000)

In [11]:
X = df.drop("volume", axis=1)
Y = df["volume"] 

In [12]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [13]:
Y.head()

0    2163600.0
1    2386400.0
2    2489500.0
3    2006300.0
4    1408600.0
Name: volume, dtype: float64

In [14]:
Y = Y.divide(1000000)

In [15]:
Y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.3,random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
import numpy as np
x_train = np.array(xtrain, dtype="float32")
y_train = np.array(ytrain, dtype="float32")
x_test = np.array(xtest, dtype="float32")
y_test = np.array(ytest, dtype="float32")

In [18]:
x_train

array([[ 27.77   ,  27.65   ,  27.41   ,  27.84   ],
       [ 12.94   ,  13.33   ,  12.88   ,  13.37   ],
       [ 18.57   ,  18.89   ,  18.4    ,  18.97   ],
       ...,
       [627.18   , 623.99   , 621.54004, 627.83997],
       [125.19   , 123.     , 121.95   , 125.91   ],
       [113.04   , 113.32   , 111.65   , 115.36   ]], dtype=float32)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [19]:
from sklearn.preprocessing import Normalizer
N = Normalizer()
x_train=N.fit_transform(x_train)
x_test=N.transform(x_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [20]:
W = tf.Variable(tf.random_normal(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
W

<tf.Variable 'Weights:0' shape=(4, 1) dtype=float32_ref>

2.Define a function to calculate prediction

In [22]:
y_pred = tf.add(tf.matmul(x_train,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [23]:
loss = tf.reduce_mean(tf.square(y_pred-y_train),name='mse')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [24]:
train_gd = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [25]:
x = tf.placeholder(shape=[None,4], dtype=tf.float32)  # x place holder
y = tf.placeholder(shape=[None], dtype=tf.float32) # y place holder

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
for epoch in range(training_epochs):
    _, train_loss = sess.run([train_gd,loss],feed_dict={x:x_train, y:y_train})  
    print ('Training loss at every step: ', epoch, ' is ', train_loss)

Training loss at every step:  0  is  224.76375
Training loss at every step:  1  is  218.14021
Training loss at every step:  2  is  213.01097
Training loss at every step:  3  is  209.03882
Training loss at every step:  4  is  205.96277
Training loss at every step:  5  is  203.58069
Training loss at every step:  6  is  201.736
Training loss at every step:  7  is  200.30745
Training loss at every step:  8  is  199.20117
Training loss at every step:  9  is  198.34444
Training loss at every step:  10  is  197.68102
Training loss at every step:  11  is  197.16722
Training loss at every step:  12  is  196.76938
Training loss at every step:  13  is  196.46126
Training loss at every step:  14  is  196.22266
Training loss at every step:  15  is  196.03787
Training loss at every step:  16  is  195.89478
Training loss at every step:  17  is  195.784
Training loss at every step:  18  is  195.69815
Training loss at every step:  19  is  195.63173
Training loss at every step:  20  is  195.58025
Traini

In [27]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
for epoch in range(training_epochs):
    _, train_loss = sess.run([train_gd,loss],feed_dict={x:x_train, y:y_train})  
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  238.91086
Training loss at step:  5  is  207.5208
Training loss at step:  10  is  198.77838
Training loss at step:  15  is  196.34348
Training loss at step:  20  is  195.66531
Training loss at step:  25  is  195.47647
Training loss at step:  30  is  195.42386
Training loss at step:  35  is  195.40921
Training loss at step:  40  is  195.40514
Training loss at step:  45  is  195.404
Training loss at step:  50  is  195.40369
Training loss at step:  55  is  195.4036
Training loss at step:  60  is  195.40356
Training loss at step:  65  is  195.40356
Training loss at step:  70  is  195.40356
Training loss at step:  75  is  195.40356
Training loss at step:  80  is  195.40356
Training loss at step:  85  is  195.40356
Training loss at step:  90  is  195.40356
Training loss at step:  95  is  195.40356


### Get the shapes and values of W and b

In [28]:
sess.run(W)

array([[ 0.6688872 ],
       [ 2.1279469 ],
       [ 1.7080756 ],
       [-0.04335737]], dtype=float32)

In [29]:
sess.run(b)

array([3.2980797], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [30]:
y_test_pred = tf.add(tf.matmul(x_test,W),b,name='output')

In [31]:
sess.run(y_test_pred)

array([[5.519988 ],
       [5.5249915],
       [5.5232267],
       [5.523267 ],
       [5.525137 ],
       [5.5225563],
       [5.5266466],
       [5.5196285],
       [5.514045 ],
       [5.5241623],
       [5.52149  ],
       [5.5249043],
       [5.523815 ],
       [5.524747 ],
       [5.510103 ],
       [5.5159607],
       [5.523132 ],
       [5.499506 ],
       [5.5192   ],
       [5.5107794],
       [5.523558 ],
       [5.5239058],
       [5.523819 ],
       [5.5140243],
       [5.526622 ],
       [5.497551 ],
       [5.5265884],
       [5.520812 ],
       [5.5174956],
       [5.5117226],
       [5.5245733],
       [5.520252 ],
       [5.5111036],
       [5.520823 ],
       [5.5088644],
       [5.523848 ],
       [5.523883 ],
       [5.52378  ],
       [5.521846 ],
       [5.522073 ],
       [5.522361 ],
       [5.523384 ],
       [5.511075 ],
       [5.5230465],
       [5.5224338],
       [5.517988 ],
       [5.5244074],
       [5.524369 ],
       [5.5136456],
       [5.5277414],


In [32]:
sess.close()

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [33]:
iris = pd.read_csv('Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [34]:
iris.drop("Id", axis=1, inplace=True)

In [35]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [36]:
iris['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [37]:
# pd.get_dummies(iris, columns=['Species'], drop_first = True)
dummy = pd.get_dummies(iris['Species'])
dummy.sample(10)

,Iris-setosa,Iris-versicolor,Iris-virginica
39,1,0,0
26,1,0,0
101,0,0,1
85,0,1,0
65,0,1,0
51,0,1,0
55,0,1,0
111,0,0,1
7,1,0,0
8,1,0,0


### Splitting the data into feature set and target set

In [38]:
X = iris.drop("Species", axis=1)
Y = dummy

In [39]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [40]:
Y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [41]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

Using TensorFlow backend.


### Model Training 

In [42]:
model.fit(X, Y, epochs=100, batch_size=5) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
150/150 [==============================] - 1s 6ms/step - loss: 1.2699 - acc: 0.3000
Epoch 2/100
150/150 [==============================] - 0s 580us/step - loss: 0.8670 - acc: 0.6200
Epoch 3/100
150/150 [==============================] - 0s 727us/step - loss: 0.7325 - acc: 0.8000
Epoch 4/100
150/150 [==============================] - 0s 793us/step - loss: 0.6516 - acc: 0.7933 0s - loss: 0.7025 - acc: 0.72
Epoch 5/100
150/150 [==============================] - 0s 1ms/step - loss: 0.5861 - acc: 0.8200
Epoch 6/100
150/150 [==============================] - 0s 567us/step - loss: 0.5411 - acc: 0.8733
Epoch 7/100
150/150 [==============================] - 0s 613us/step - loss: 0.4987 - acc: 0.8667
Epoch 8/100
150/150 [==============================] - 0s 607us/step - loss: 0.4760 - acc: 0.8667
Epoch 9/100
150/150 [==============================] - 0s 587us/step - loss: 0.4351 - acc: 0.8867
Epoch 10/100
150/150 [======================

150/150 [==============================] - 0s 673us/step - loss: 0.1221 - acc: 0.9667
Epoch 81/100
150/150 [==============================] - 0s 833us/step - loss: 0.1134 - acc: 0.9533
Epoch 82/100
150/150 [==============================] - 0s 613us/step - loss: 0.1187 - acc: 0.9533
Epoch 83/100
150/150 [==============================] - 0s 487us/step - loss: 0.1340 - acc: 0.9600
Epoch 84/100
150/150 [==============================] - 0s 527us/step - loss: 0.1071 - acc: 0.9600
Epoch 85/100
150/150 [==============================] - 0s 487us/step - loss: 0.1114 - acc: 0.9600
Epoch 86/100
150/150 [==============================] - 0s 493us/step - loss: 0.1035 - acc: 0.9600
Epoch 87/100
150/150 [==============================] - 0s 767us/step - loss: 0.1132 - acc: 0.9667
Epoch 88/100
150/150 [==============================] - 0s 540us/step - loss: 0.1231 - acc: 0.9533
Epoch 89/100
150/150 [==============================] - 0s 513us/step - loss: 0.1095 - acc: 0.9667
Epoch 90/100
150/150 [=

### Model Prediction

In [44]:
scores = model.evaluate(X, Y)
scores[1]*100

150/150 [==============================] - 0s 60us/step


97.33333333333334

### Save the Model

In [ ]:
import pickle
pickle.dump(model, open("model.pkl","wb" , ))
#model = pickle.load(open("model.pkl", "rb"))
##"rb" : this means read mode

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?